<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy8_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import random
from copy import deepcopy
from queue import PriorityQueue
from collections import defaultdict

# Klasa Grafy

In [ ]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph
        self.weighted = False

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) == 2: # two words -- unweighed graph
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
            elif len(words) > 2: # more than two words - labeled graph
                graph.weighted = True
                if directed:
                    graph.add_arc([words[0], words[1]])
                    graph.graph[words[0]][-1] = (words[1], words[2]) # process label
                else:
                    graph.add_edge([words[0], words[1]])
                    graph.graph[words[0]][-1] = (words[1], words[2])
                    graph.graph[words[1]][-1] = (words[0], words[2])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            rand_graph.add_node(i)
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr

    def Prufer(self):
      """
      Kod Prufera drzewa - zwrócony jako napis
      Wymagane, aby graf był drzewem.
      Zwraca pusty napis dla drzew o mniej niż 3 wierzchołkach.
      """
      tr = deepcopy(self.graph)   # będziemy psuć graf
      code = ""
      for i in range(len(self.graph) - 2):
        for x in sorted(tr):    # po kolei przeglądam nieusunięte wierzchołki
          if len(tr[x]) == 1:   # najmniejszy liść
            break
        v = tr[x][0]  # sąsiad najmniejszego x
        code = code + f"{v} "
        tr[v].remove(x)   # usuwam x z listy sąsiadów v
        tr.pop(x)         # usuwam x z drzewa
      return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Tworzy drzewo na podstawie kodu Prufera.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]   # kod zamieniamy na listę liczb
        n = len(clist) + 2    # liczba wierzchołków
        vert = [v for v in range(1, n+1)]  # lista liczb od 1 do n
        for v in vert:
          tree.add_node(v)
        for i in range(n-2):
          for x in vert:
            if not x in clist:    # najmniejszy liść
              break
          v = clist.pop(0)    # usuwam pierwszy element listy (sąsiad x)
          tree.add_edge((x, v))
          vert.remove(x)
        tree.add_edge(vert)
        return tree

    def ConnectedComponents(self):
      """
      Znajduje spójne składowe w grafie nieskierowanym
      Jako wynik zwraca listę zbiorów wierzchołków
      Uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu
      """
      def DFS(u):
        """
        Przeszukiwanie w głąb
        """
        for w in self.graph[u]:
          if not w in VT[0]:  # w jeszcze nie odwiedzony
            VT[0].add(w)      # już odwiedzony
            VT[-1].add(w)     # w ostatniej spójnej składowej
            DFS(w)
      """
      VT - lista zbiorów VT[i] dla i > 0 - lista wierzchołków spójnych składowych
      dla i = 0 - lista wszystkich odwiedzonych wierzchołków
      """
      VT = [set([])]
      for v in self.graph:
        if v not in VT[0]:
          VT[0].add(v)
          VT.append(set([v]))   # zaczątek nowej spójnej składowej
          DFS(v)
      return VT

    def Connected_components_graphs(self):
      """
      Spójne składowe jako grafy
      """
      VT=self.ConnectedComponents()
      noCc=len(VT)-1  #liczba spojnych skladowych
      graphs=[]  #lista do przechowywania grafow reprezentujacych spojne skladowe
      for i in range(noCc):  #iterujemy przez wszystkie spojne skladowe
          graph_i={}    #dla każdej spójnej składowej tworzymy nowy graf graph_i, który jest podgrafem oryginalnego grafu,
          for v in VT[i+1]:     #zawierającym tylko wierzchołki z bieżącej spójnej składowej
              graph_i[v]=self.graph[v].copy()
          graphs.append(Graph.from_dict(graph_i))  #dodajemy stworzone grafy do listy
      return graphs

    def Distance(self, v):
      """
      Znajduje i zwraca jako wektor słownik odległości od wierzchołka v
      do wierzchołków w tej samej spójnej składowej co v
      """
      dist = {v:0} # zalążek słownika odległości
      kolejka = [v]
      while len(kolejka) > 0:
        u = kolejka.pop(0)
        for w in self.graph[u]:
          if not w in dist:
            dist[w] = dist[u] + 1
            kolejka.append(w)
      return dist


    def Bellman_Ford(self, s, t):
      if not self.weighted: # jak graf nie jest ważony - zwróć nic
        return None, None
      # Init
      dist = {}
      pred = {}
      for v in self.graph:
        dist[v] = 2**31 # substytut nieskończoności
        pred[v] = None
      dist[s] = 0
      for i in range(len(self.graph)-1):
        for u in self.graph:
          for v, w, _ in self.graph[u]:  # relax
            if dist[v] > dist[u] + int(w):
              dist[v] = dist[u] + int(w)
              pred[v] = u
      for u in self.graph:
        for v,w, _ in self.graph[u]:
          if dist[v] > dist[u] + int(w):
            print(f"graf zawiera cykl o ujemnej wadze {u}, {v}")
      return dist, pred


#zamienić poniższy kod, tak żeby używał bellmana-forda, żeby przyjmował
#koszt przepływu grafu skierowanego,tak aby przeszedł z punktu s do t wszystkimi możliwymi ścieżkami i minimalizował koszt (podczas, gdy cała
#przepustowość przepływu została wykorzystana może przejść w drugą stronę z kosztem ujemnym wykorzystując całą pierwotną przepustowość)

#Cały czas najtańsza ścieżka, zwykły bellman-ford, belmanem fordem minimalizujemty graf po kosztach



    def MaxFlow_MinCost(self, s, t):
        """
        Algorytm na maksymalny przepływ z minimalnym kosztem
        """
        if not self.weighted:  # jeśli graf nie jest ważony - zwróć nic
            return None, None

        n = len(self.graph)
        network = {u: [] for u in self.graph}
        for u in self.graph:
            for v, cap, cos in self.graph[u]:
                if int(cap) > 0:
                    network[u].append((v, int(cap), cos))
                    if v not in network:
                        network[v] = []
                    network[v].append((u, 0, -cos))
        print(network)

        flow = np.zeros((n, n))
        nodes = list(network.keys())
        print(nodes)

        cap_s=0
        for i in range(len(network['s'])):
            cap_s += network['s'][i][1]

        def find_path():
            dist, pred = self.Bellman_Ford(s, t)
            if dist[t] == float('inf'):
                return None
            path = []
            u = t
            while u is not None:
                path.append(u)
                u = pred[u]
            path.reverse()
            if path[0] != s:
                return None
            return path

        total_flow = 0
        total_cost = 0



        while cap_s > 0:

           # if capacity s to a > 0 then
            path = find_path()
            #elif capacity s to a = 0 the delete the path from s to a and search for the path again
            if path is None:
                break

            # Znaleziono ścieżkę powiększającą
            print(path)
            capacities = []

            for i in range(len(path)-1): #Długość ścieżki - 1
               for j in range(len(network[path[i]])): #Długość składowych wpisu słownika
                  if(network[path[i]][j][0] == path[i+1]): #Jeśli wpis o literze path[i] w ścieżce na j pozycji ma literę path[i+1]
                    capacities.append(network[path[i]][j][1])

            flow = min(capacities)
            #print(flow)
            total_flow += flow
            capacities.clear()

            if flow > 0:
              for i in range(len(path) - 1):
                for j in range(len(network[path[i]])):
                    print(i,j)
                    if (network[path[i]][j][0] == path[i+1]):
                        network[path[i]][j] = (network[path[i]][j][0], network[path[i]][j][1] - flow, network[path[i]][j][2])
                        network[path[j]][i] = (network[path[j]][i][0], network[path[j]][i][1] + flow, network[path[j]][i][2])
                        print(network)
              total_cost += flow * network[path[i]][j][2]
              print(total_cost)
            else:
              break #jak nadpisać pojedyncze ścieżki, czy zadziała dodatnie do innego slownika


        return total_flow, total_cost
h

    def MaxFlow_EK(self, s, t):
        """
        Algorytm Edmondsa-Karpa na maksymalny przepływ
        """
        if not self.weighted: # jak graf nie jest ważony - zwróć nic
          return None, None
        n = len(self.graph)
        #przygotowanie - sieć przepustowość
        network = {}
        for u in self.graph:
          network[u] = []
        for u in self.graph:
          for (v, w) in self.graph[u]:
            if int(w) > 0:
              network[u].append((v, int(w)))
              network[v].append((u, 0))
        flow = np.zeros((n,n))
        nodes = [*network.keys()]
        while True:
          # ścieżki powiększające s - > t
          pred = {s:None} # zalążek słownika poprzedników
          cap = {s:2^31}  # zalążek słownika przepustowości
          kolejka = [s]
          while len(kolejka) > 0:
            u = kolejka.pop(0)
            for (v, w) in network[u]:
              if (not v in pred) and w + flow[nodes.index(v), nodes.index(u)] > 0:
                pred[v] = u
                cap[v] = min(cap[u], w + flow[nodes.index(v), nodes.index(u)])
                kolejka.append(v)
              if v == t:
                break
          if not t in pred:
            break # koniec - nie ma s->t-ścieżki
          else:
            u = t
            c = cap[t]
            while u != s:
              v = pred[u]
              flow[nodes.index(v), nodes.index(u)] += c
              flow[nodes.index(u), nodes.index(v)] -= c
              u = v

        return nodes, flow

# Przykłady wykorzystania

In [ ]:
!wget https://raw.githubusercontent.com/pgordin/OptDisc2024/main/wagi2.txt

--2024-06-17 07:45:31--  https://raw.githubusercontent.com/pgordin/OptDisc2024/main/wagi2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67 [text/plain]
Saving to: ‘wagi2.txt’

wagi2.txt           100%[===================>]      67  --.-KB/s    in 0s      

2024-06-17 07:45:31 (1.07 MB/s) - ‘wagi2.txt’ saved [67/67]



In [ ]:
%cat wagi2.txt

1 2 3
1 3 8
1 5 -4
4 1 2
3 2 4
2 4 1
2 5 7
4 3 -5 
5 4 6 


In [ ]:
wgraph = Graph.from_edges("wagi2.txt", directed = 1)
print(wgraph)

1: ('2', '3') ('3', '8') ('5', '-4')
2: ('4', '1') ('5', '7')
3: ('2', '4')
5: ('4', '6')
4: ('1', '2') ('3', '-5')



In [ ]:
dist,pred = wgraph.Bellman_Ford('1')

In [ ]:
print(dist)

{'1': 0, '2': 1, '3': -3, '5': -4, '4': 2}


In [ ]:
print(pred)

{'1': None, '2': '3', '3': '4', '5': '1', '4': '5'}


# Maksymalny przepływ

In [ ]:
graph1 = Graph.from_edges("flow1.txt", directed = 1)
print(graph1)

s: ('a', '6') ('b', '4')
a: ('b', '5') ('c', '2') ('d', '3')
b: ('c', '6')
c: ('a', '2') ('d', '3') ('t', '4')
d: ('t', '6')
t:



In [ ]:
nodes, flow = graph1.MaxFlow_EK("s", "t")

In [ ]:
print(nodes)

['s', 'a', 'b', 'c', 'd', 't']


In [ ]:
print(flow)

[[ 0.  6.  4.  0.  0.  0.]
 [-6.  0.  1.  2.  3.  0.]
 [-4. -1.  0.  5.  0.  0.]
 [ 0. -2. -5.  0.  3.  4.]
 [ 0. -3.  0. -3.  0.  6.]
 [ 0.  0.  0. -4. -6.  0.]]


In [ ]:
graph = {
    's': [('a', 10, 4), ('d', 10, 4)],
    'a': [('b', 5, 20), ('e', 8, 2)],
    'b': [('c', 13, 2)],
    'c': [('t', 10, 4)],
    'd': [('e', 5, 1), ('b', 8, 2)],
    'e': [('t', 10, 4)],
    't': []
}

In [ ]:
g = Graph(graph)
g.weighted = True

In [ ]:
total_flow, total_cost = g.MaxFlow_MinCost('s', 't')

{'s': [('a', 10, 4), ('d', 10, 4)], 'a': [('s', 0, -4), ('b', 5, 20), ('e', 8, 2)], 'b': [('a', 0, -20), ('c', 13, 2), ('d', 0, -2)], 'c': [('b', 0, -2), ('t', 10, 4)], 'd': [('s', 0, -4), ('e', 5, 1), ('b', 8, 2)], 'e': [('a', 0, -2), ('d', 0, -1), ('t', 10, 4)], 't': [('c', 0, -4), ('e', 0, -4)]}
['s', 'a', 'b', 'c', 'd', 'e', 't']
['s', 'd', 'e', 't']
0 0
0 1
{'s': [('a', 10, 4), ('d', 5, 4)], 'a': [('s', 0, -4), ('b', 5, 20), ('e', 8, 2)], 'b': [('a', 0, -20), ('c', 13, 2), ('d', 0, -2)], 'c': [('b', 0, -2), ('t', 10, 4)], 'd': [('s', 5, -4), ('e', 5, 1), ('b', 8, 2)], 'e': [('a', 0, -2), ('d', 0, -1), ('t', 10, 4)], 't': [('c', 0, -4), ('e', 0, -4)]}
1 0
1 1
{'s': [('a', 10, 4), ('d', 5, 4)], 'a': [('s', 0, -4), ('b', 5, 20), ('e', 8, 2)], 'b': [('a', 0, -20), ('c', 13, 2), ('d', 0, -2)], 'c': [('b', 0, -2), ('t', 10, 4)], 'd': [('s', 5, -4), ('e', 5, 1), ('b', 8, 2)], 'e': [('a', 0, -2), ('d', 0, -1), ('t', 10, 4)], 't': [('c', 0, -4), ('e', 0, -4)]}
1 2
2 0
2 1
2 2
{'s': [('a', 